In [1]:
from sfcn import SFCN
import numpy as np
import pandas as pd
from volumedatagenerator import VolumeDataGeneratorRegression
import matplotlib.pyplot as plt

2021-12-19 12:10:03.857960: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# col_list = pd.read_csv('fields/subcortical_volumes.csv')['field id'].to_list()

# columns=['path']
# for col in col_list:
#     columns.append(str(col)+'-2.0')

In [3]:
train_df = pd.read_csv('csv/split_train.csv', index_col='eid').dropna()
valid_df = pd.read_csv('csv/split_valid.csv', index_col='eid').dropna()
test_df = pd.read_csv('csv/split_test.csv', index_col='eid').dropna()

In [4]:
name = 'sfcn_avg_norm'
batch_size = 6
gpu_num = 6
cpu_workers = 6
epochs_num = 64

index=2

In [5]:
input_dim = [182, 218, 182]
num_output = len(train_df.columns)-1

In [6]:
train_gen = VolumeDataGeneratorRegression(
    sample_df=train_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_preprocessing='quantile')

scaler_instance = train_gen.get_scaler_instance()

In [7]:
valid_gen = VolumeDataGeneratorRegression(
    sample_df=valid_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_scaler=scaler_instance,
    shuffle=False)

In [8]:
test_gen = VolumeDataGeneratorRegression(
    sample_df=test_df, 
    batch_size=batch_size, 
    #num_reg_classes=num_output, 
    dim=input_dim,
    output_scaler=scaler_instance,
    shuffle=False
)

In [9]:
model = SFCN(
        input_dim=[182, 218, 182, 1], 
        output_dim=num_output,
        conv_num_filters=[32, 64, 128, 256, 256, 64], 
        conv_kernel_sizes=[3, 3, 3, 3, 3, 1], 
        conv_strides=[1, 1, 1, 1, 1, 1],
        conv_padding=['same', 'same', 'same', 'same', 'same', 'valid'],
        pooling_size=[2, 2, 2, 2, 2],
        pooling_type=['avg_pool', 'avg_pool', 'avg_pool', 'avg_pool', 'avg_pool'],
        batch_norm=True,
        dropout=False,
        softmax=False,
        gpu_num=gpu_num,
        name=name+'_'+str(index)
        )
model.compile(learning_rate=3e-4)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5')


2021-12-19 12:10:17.309791: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-12-19 12:10:17.312413: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-12-19 12:10:17.660131: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-12-19 12:10:17.663059: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-12-19 12:10:17.665899: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:07:0

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [10]:
model.train_generator(train_gen, valid_gen, batch_size=batch_size, epochs=epochs_num, workers=cpu_workers)

2021-12-19 12:10:26.819722: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_1924"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  va

Epoch 1/64
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 26 all-reduces with algorithm = nccl, num_packs = 1


2021-12-19 12:10:44.207343: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-12-19 12:10:45.645983: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2021-12-19 12:10:45.960938: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-12-19 12:10:48.840274: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


1031/1031 [==============================] - ETA: 0s - loss: 0.1071 - mae: 0.2730

2021-12-19 12:23:35.511694: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_23353"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  v

1031/1031 [==============================] - 980s 775ms/step - loss: 0.1071 - mae: 0.2729 - val_loss: 0.1958 - val_mae: 0.3623
Epoch 2/64
1031/1031 [==============================] - 820s 793ms/step - loss: 0.0834 - mae: 0.2496 - val_loss: 2.7413 - val_mae: 1.5146
Epoch 3/64
1031/1031 [==============================] - 833s 806ms/step - loss: 0.0813 - mae: 0.2460 - val_loss: 0.5720 - val_mae: 0.5185
Epoch 4/64
1031/1031 [==============================] - 780s 754ms/step - loss: 0.0802 - mae: 0.2439 - val_loss: 0.8174 - val_mae: 0.5828
Epoch 5/64
1031/1031 [==============================] - 783s 757ms/step - loss: 0.0784 - mae: 0.2408 - val_loss: 0.1790 - val_mae: 0.3004
Epoch 6/64
1031/1031 [==============================] - 805s 779ms/step - loss: 0.0773 - mae: 0.2388 - val_loss: 0.2564 - val_mae: 0.3520
Epoch 7/64
1031/1031 [==============================] - 832s 805ms/step - loss: 0.0752 - mae: 0.2350 - val_loss: 0.5199 - val_mae: 0.5013
Epoch 8/64
1031/1031 [=======================

In [11]:
model.load_weights('weights/checkpoint_' + name + '_' + str(index))
model.evaluate_generator(valid_gen, batch_size, filename=name + '_val', workers=cpu_workers)
model.evaluate_generator(test_gen, batch_size, filename=name + '_test', workers=cpu_workers)

2021-12-19 19:59:08.878652: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_160053"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  